In [1]:
import os
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report
import torchvision.models as models

import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim
import re

# 读取数据

In [2]:
files_path = './FBDQA2021A_MMP_Challenge/train_data.csv'
df =  pd.read_csv(files_path)
df = df.sort_values(by=['sym','date',"date_time"])

In [3]:
# 价格+1（从涨跌幅还原到对前收盘价的比例）
df['bid1'] = df['bid1']+1
df['bid2'] = df['bid2']+1
df['bid3'] = df['bid3']+1
df['bid4'] = df['bid4']+1
df['bid5'] = df['bid5']+1
df['ask1'] = df['ask1']+1
df['ask2'] = df['ask2']+1
df['ask3'] = df['ask3']+1
df['ask4'] = df['ask4']+1
df['ask5'] = df['ask5']+1

# 量价组合
df['spread1'] =  df['ask1'] - df['bid1']
df['spread2'] =  df['ask2'] - df['bid2']
df['spread3'] =  df['ask3'] - df['bid3']
df['mid_price1'] =  df['ask1'] + df['bid1']
df['mid_price2'] =  df['ask2'] + df['bid2']
df['mid_price3'] =  df['ask3'] + df['bid3']
df['weighted_ab1'] = (df['ask1'] * df['bsize1'] + df['bid1'] * df['asize1']) / (df['bsize1'] + df['asize1'])
df['weighted_ab2'] = (df['ask2'] * df['bsize2'] + df['bid2'] * df['asize2']) / (df['bsize2'] + df['asize2'])
df['weighted_ab3'] = (df['ask3'] * df['bsize3'] + df['bid3'] * df['asize3']) / (df['bsize3'] + df['asize3'])

df['vol1_rel_diff']   = (df['bsize1'] - df['asize1']) / (df['bsize1'] + df['asize1'])
df['volall_rel_diff'] = (df['bsize1'] + df['bsize2'] + df['bsize3'] + df['bsize4'] + df['bsize5'] \
                 - df['asize1'] - df['asize2'] - df['asize3'] - df['asize4'] - df['asize5'] ) / \
                 ( df['bsize1'] + df['bsize2'] + df['bsize3'] + df['bsize4'] + df['bsize5'] \
                 + df['asize1'] + df['asize2'] + df['asize3'] + df['asize4'] + df['asize5'] )

df['amount'] = df['amount_delta'].map(np.log1p)

df['bid1'] = df['bid1']-1
df['bid2'] = df['bid2']-1
df['bid3'] = df['bid3']-1
df['bid4'] = df['bid4']-1
df['bid5'] = df['bid5']-1
df['ask1'] = df['ask1']-1
df['ask2'] = df['ask2']-1
df['ask3'] = df['ask3']-1
df['ask4'] = df['ask4']-1
df['ask5'] = df['ask5']-1


In [4]:
feature_col_names = ['bid1','bsize1',
                     'bid2','bsize2',
                     'bid3','bsize3',
                     'bid4','bsize4',
                     'bid5','bsize5',
                     'ask1','asize1',
                     'ask2','asize2',
                     'ask3','asize3',
                     'ask4','asize4',
                     'ask5','asize5',
                     'spread1','mid_price1',
                     'spread2','mid_price2',
                     'spread3','mid_price3',
                     'weighted_ab1','weighted_ab2','weighted_ab3','amount',
                     'vol1_rel_diff','volall_rel_diff'
                    ]
label_col_name = ['label_40']

In [5]:
n = len(df['date'].unique())

# ##划分训练/测试集
train_nums = int(n*0.8 + 0.5)
val_nums = int(n*0.1 + 0.5)
print(f'train_date_nums: {train_nums}, val_date_nums: {val_nums}, test_date_nums: {n-train_nums-val_nums}')
train_data = df[df.date < train_nums]
val_data = df[(df.date >= train_nums) & (df.date < train_nums+val_nums)]
test_data = df[df.date >= train_nums+val_nums]

train_date_nums: 63, val_date_nums: 8, test_date_nums: 8


## GPU准备

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## 准备dataset

In [13]:
def data_transform(X, T):
    [N, D] = X.shape
    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = X[i - T:i, :]
    return dataX.tolist()

In [14]:
class Dataset(data.Dataset):
    def __init__(self, data, num_classes, T):
        self.T = T
        
        dates = data['date'].unique()
        syms = data['sym'].unique()
        data_result = []
        label_result = []
        for sym in syms:
            for date in dates:
                sym_date_df = data[(data.sym == sym)&(data.date == date)].copy()
                if len(sym_date_df) == 0:
                    continue
                    
                data_result += data_transform(sym_date_df[feature_col_names].values, self.T)
                label_result += sym_date_df[label_col_name][T - 1:].values.tolist()
                
        self.x = torch.tensor(data_result).to(torch.float32).unsqueeze(1).to(device)
        label_result = np.array(label_result).reshape(-1)
        self.y = torch.tensor(label_result.astype(np.int64)).to(device)
        
        self.length = len(self.x)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [15]:
dataset_val   = Dataset(data=val_data, num_classes=3, T=100)
# dataset_test  = Dataset(data=test_data, num_classes=3, T=100)
# dataset_train = Dataset(data=train_data, num_classes=3, T=100)

In [16]:
dataset_test  = Dataset(data=test_data, num_classes=3, T=100)

In [17]:
dataset_train = Dataset(data=train_data, num_classes=3, T=100)

In [18]:
# import pickle

# # 获取当前文件所在路径的上一层路径
# parent_dir = os.path.dirname(os.path.abspath(os.getcwd()))  # __file__表示当前文件名，可以用其他文件名替换
# grandparent_dir = os.path.dirname(os.path.abspath(parent_dir))
# # 输出上一层路径
# print(parent_dir)
# print(grandparent_dir)

# # 保存 Dataset 对象到文件中
# with open(grandparent_dir+'/dataset_train_diff_1channel_40_new.pickle', 'wb') as f:
#     pickle.dump(dataset_train, f)
# with open(grandparent_dir+'/dataset_test_diff_1channel_40_new.pickle', 'wb') as f:
#     pickle.dump(dataset_test, f)
# with open(grandparent_dir+'/dataset_val_diff_1channel_40_new.pickle', 'wb') as f:
#     pickle.dump(dataset_val, f)

In [19]:
# import pickle

# # 获取当前文件所在路径的上一层路径
# parent_dir = os.path.dirname(os.path.abspath(os.getcwd()))  # __file__表示当前文件名，可以用其他文件名替换
# grandparent_dir = os.path.dirname(os.path.abspath(parent_dir))
# # 输出上一层路径
# print(parent_dir)
# print(grandparent_dir)

# # 从文件中加载 Dataset 对象
# with open(grandparent_dir+'/dataset_train_diff_1channel_60.pickle', 'rb') as f:
#     dataset_train = pickle.load(f)
# with open(grandparent_dir+'/dataset_val_diff_1channel_60.pickle', 'rb') as f:
#     dataset_val = pickle.load(f)
# with open(grandparent_dir+'/dataset_test_diff_1channel_60.pickle', 'rb') as f:
#     dataset_test = pickle.load(f)

In [20]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader   = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader  = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape, dataset_train.x.requires_grad, dataset_train.y.requires_grad,)
print(dataset_val.x.shape, dataset_val.y.shape, dataset_val.x.requires_grad, dataset_val.y.requires_grad,)
print(dataset_test.x.shape, dataset_test.y.shape, dataset_test.x.requires_grad, dataset_test.y.requires_grad,)

torch.Size([2348108, 1, 100, 32]) torch.Size([2348108]) False False
torch.Size([306022, 1, 100, 32]) torch.Size([306022]) False False
torch.Size([309921, 1, 100, 32]) torch.Size([309921]) False False


In [21]:
print(dataset_train.x.device)
print(dataset_train.y.device)

cuda:0
cuda:0


# 定义模型

In [22]:
from mmpc.model import Res50Transformer

In [23]:
model = Res50Transformer(num_classes = 3)
model.to(device);

# 训练模型

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [25]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    best_test_loss = np.inf
    best_test_epoch = 0
    step = 0
    save_step = 20000
    num_data_points = len(train_loader.dataset)
    train_losses = np.zeros((int(num_data_points/save_step)+1)*epochs)
    test_losses = np.zeros((int(num_data_points/save_step)+1)*epochs)
    
    old_checkpoint_dir = 'ResNetcheckpoints_label40_new'

    # Load pretrained weights
    Res50checkpoints = [f for f in os.listdir(old_checkpoint_dir) if re.match(r'ResNet_\d+.pth', f)]
    pretrained_dir = max(Res50checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    print(f'Loaded model from {pretrained_dir}')
    pretrained_dict = torch.load('./ResNetcheckpoints_label40_new/'+pretrained_dir)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if (k in model_dict and (k.startswith('conv1.') or k.startswith('resnet50.')))}
    print("Inherited parameters from dict {}".format(pretrained_dict.keys()))
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    
    checkpoint_dir = 'Res50Transformer_checkpoints_label40_new'
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    # Load the latest checkpoint if available
    checkpoints = [f for f in os.listdir(checkpoint_dir) if re.match(r'Res50Transformer_\d+.pth', f)]
    try:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
        step = int(latest_checkpoint.split('_')[-1].split('.')[0])
        model.load_state_dict(torch.load(os.path.join(checkpoint_dir, latest_checkpoint)))
        print(f'Loaded model from {latest_checkpoint}')
    except:
        torch.save(model.state_dict(), os.path.join(checkpoint_dir, 'Res50Transformer_0.pth'))
        print(f'Initialized new model and saved as {os.path.join(checkpoint_dir, "Res50Transformer_0.pth")}')

    for it in tqdm(range(epochs)):
        # optimizer.lr = optimizer.lr * (0.98**(step/save_step))
        model.train()
        train_loss = []
        t0 = datetime.now()

        for inputs, targets in train_loader:
            optimizer.zero_grad()            
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            
            train_loss.append(loss.item())

            # Save model every 150000 steps
            step += 1
            if step % save_step == 0:
                
                torch.save(model.state_dict(), os.path.join(checkpoint_dir, f'Res50Transformer_{step}.pth'))
                print(f'Res50Transformer Model saved at step {step}')

        train_loss = np.mean(train_loss)

        model.eval()
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)
        current_lr = optimizer.state_dict()['param_groups'][0]['lr']

        train_losses[it] = train_loss
        test_losses[it] = test_loss

        if test_loss < best_test_loss:
            torch.save(model.state_dict(), f'Res50Transformer_best_val_model.pth')
            best_test_loss = test_loss
            best_test_epoch = it
            print('Res50Transformer model saved')

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f},  Learning Rate: {current_lr:.9f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')

    torch.save(model.state_dict(), f'final_Res50Transformer_model.pth')
    return train_losses, test_losses

In [26]:
train_losses, val_losses = batch_gd(model, criterion, optimizer, 
                                    train_loader, val_loader, epochs=20)

Loaded model from ResNet_100000.pth
Inherited parameters from dict dict_keys(['resnet50.conv1.weight', 'resnet50.bn1.weight', 'resnet50.bn1.bias', 'resnet50.bn1.running_mean', 'resnet50.bn1.running_var', 'resnet50.bn1.num_batches_tracked', 'resnet50.layer1.0.conv1.weight', 'resnet50.layer1.0.bn1.weight', 'resnet50.layer1.0.bn1.bias', 'resnet50.layer1.0.bn1.running_mean', 'resnet50.layer1.0.bn1.running_var', 'resnet50.layer1.0.bn1.num_batches_tracked', 'resnet50.layer1.0.conv2.weight', 'resnet50.layer1.0.bn2.weight', 'resnet50.layer1.0.bn2.bias', 'resnet50.layer1.0.bn2.running_mean', 'resnet50.layer1.0.bn2.running_var', 'resnet50.layer1.0.bn2.num_batches_tracked', 'resnet50.layer1.0.conv3.weight', 'resnet50.layer1.0.bn3.weight', 'resnet50.layer1.0.bn3.bias', 'resnet50.layer1.0.bn3.running_mean', 'resnet50.layer1.0.bn3.running_var', 'resnet50.layer1.0.bn3.num_batches_tracked', 'resnet50.layer1.0.downsample.0.weight', 'resnet50.layer1.0.downsample.1.weight', 'resnet50.layer1.0.downsample.

  0%|          | 0/20 [00:00<?, ?it/s]

Res50Transformer Model saved at step 220000


  0%|          | 0/20 [18:42<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(train_losses[0:30], label='train loss')
plt.plot(val_losses[0:30], label='validation loss')
plt.legend()

# 测试模型

In [ ]:
model = Res50Transformer(num_classes = 3)
model.to(device);
state_dict = torch.load(f'./Res50Transformer_checkpoints_label40_new/Res50Transformer_140000.pth', map_location=device)
# state_dict = torch.load(f'./checkpoints/transformer_334000.pth', map_location=device)
model.load_state_dict(state_dict)
model.eval()
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)    
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

In [ ]:
# model = torch.load('best_val_model_pytorch',map_location=device)
all_targets = []
all_predictions = []

for inputs, targets in train_loader:

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions) 
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))